<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install pandas
# !pip install nltk

In [74]:
import os
import re
import csv
import hazm
import nltk
import pickle
import zipfile
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [40]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [4]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [5]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = read_dataset_from_file()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [7]:
def display_dataset_info():
    global CATEGORIES

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for data in dataset:
        length_dict[data['category-ENG']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,80986
1,سیاسی,15810
2,جهانی,4800
3,اقتصادی,9873
4,اجتماعی,14070
5,شهری,4260
6,ورزشی,8421
7,علمی,3267
8,فرهنگی,6657
9,فناوری اطلاعات,444


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. در ادامه، به توضیح هر یک از مراحل پیش‌پردازش می‌پردازیم.
    </p>
</div>

In [8]:
class Preprocessor:

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.fix_hamza(text)
        text = self.fix_arabic_ta(text)
        text = self.fix_arabic_ye(text)
        text = self.fix_half_space_ye(text)
        text = self.fix_single_ye(text)
        text = self.fix_informal_numbers(text)
        return text

    def normalize(self, text):
        normalizer = hazm.Normalizer()
        return normalizer.normalize(text)

    def fix_hamza(self, text):
        return text.replace('ء', '')

    def fix_arabic_ta(self, text):
        return text.replace('ة', 'ه')

    def fix_arabic_ye(self, text):
        return text.replace('ي', 'ی')

    def fix_half_space_ye(self, text):
        return text.replace('\u200cی ', ' ')

    def fix_single_ye(self, text):
        return text.replace(' ی ', ' ')

    def fix_informal_numbers(self, text):
        pass


In [9]:
dataset = pd.DataFrame(dataset)
dataset

,date,title,intro,body,category-PER,category-ENG
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,"['سياست', 'سیاست\u200cخارجی']",Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,"['جهان', 'آمریکا']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,"['سياست', 'سیاست داخلی']",Politics
...,...,...,...,...,...,...
80981,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,"['زندگی', 'تندرستی']",LifeSkills
80982,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,"['زندگی', 'مهارت\u200cهای زندگی']",LifeSkills
80983,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,"['زندگی', 'پیشنهاد سفر']",LifeSkills
80984,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,"['زندگی', 'تندرستی']",LifeSkills


In [102]:
class TFIDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer(ngram_range=(1, 1))
        self.vectors = None
        self.feature_names = None
        self.df = None
    
    def fit_transform_vectoriazer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(dataset)
        self.feature_names = self.vectorizer.get_feature_names_out()
        dense = self.vectors.todense()
        denselist = dense.tolist()
        self.df = pd.DataFrame(denselist, columns=self.feature_names)
    
    def predict(self, query, dataset, k):
        query_transform = tf_idf.vectorizer.transform([query])
        query_transform = query_transform.todense()
        query_transform = query_transform.tolist()
        query_transform = query_transform[0]
        df_val = tf_idf.df.values.tolist()
        df_cosine_sim = [self.cosine_sim(query_transform, row) for row in df_val]
        self.df['query_sim'] = df_cosine_sim
        res = self.df.nlargest(k, 'query_sim').index
        print(res)
        self.df = self.df.drop(columns=['query_sim'])
        return dataset.iloc[res]
        

    def cosine_sim(self, query, row):
        return np.dot(query, row) / (np.linalg.norm(query) * np.linalg.norm(row))

    def save_model(self, path="tfidf_model.pickle"):
        pickle.dump(self, open(path, "wb"))
        

In [103]:
tf_idf = TFIDF()

In [104]:
tf_idf.fit_transform_vectoriazer(dataset.loc[:10, "body"].tolist())

In [105]:
query = "آخر آحاد"
tf_idf.predict(query, dataset, 3)

Int64Index([6, 3, 0], dtype='int64')


,date,title,intro,body,category-PER,category-ENG
6,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۱۴,ماجرای ادعای دستکاری در جداول بودجه چه بود؟,یک عضو کمیسیون تلفیق با بیان این که هیچ تصمیمی...,به گزارش همشهری آنلاین به نقل ازباشگاه خبرنگار...,"['سياست', 'مجلس']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics


In [106]:
dataset.loc[6]

date                              سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۱۴
title                 ماجرای ادعای دستکاری در جداول بودجه چه بود؟
intro           یک عضو کمیسیون تلفیق با بیان این که هیچ تصمیمی...
body            به گزارش همشهری آنلاین به نقل ازباشگاه خبرنگار...
category-PER                                    ['سياست', 'مجلس']
category-ENG                                             Politics
Name: 6, dtype: object

In [107]:
tf_idf.save_model()

In [111]:
mod = pickle.load(open('tfidf_model.pickle', 'rb'))
mod.vectors

<11x1258 sparse matrix of type '<class 'numpy.float64'>'
	with 2068 stored elements in Compressed Sparse Row format>